In [1]:
import json
import requests
import time
from typing import Dict, List, Any
import os

In [2]:
def load_restaurant_data(json_file_path: str) -> List[Dict[str, Any]]:
    """
    JSON 파일에서 레스토랑 데이터를 로드합니다.
    
    Args:
        json_file_path: JSON 파일 경로
        
    Returns:
        레스토랑 리스트
    """
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data.get('restaurants', [])

In [3]:
def format_restaurant_for_api(restaurant: Dict[str, Any]) -> Dict[str, Any]:
    """
    레스토랑 데이터를 API 요청 형식으로 변환합니다.
    
    Args:
        restaurant: 원본 레스토랑 데이터
        
    Returns:
        API 요청용 포맷된 데이터
    """
    # 메뉴 데이터 변환 (price_text 제거, price만 유지)
    formatted_menus = []
    for menu in restaurant.get('menus', []):
        if menu.get('name') and menu.get('price') is not None:
            formatted_menus.append({
                'name': menu['name'],
                'price': menu['price']
            })
    
    return {
        'id': restaurant.get('place_id', ''),
        'name': restaurant.get('name', ''),
        'category': restaurant.get('category', ''),
        'address': restaurant.get('address', ''),
        'contact': restaurant.get('contact', ''),
        'menus': formatted_menus
    }

In [4]:
def send_restaurant_to_api(restaurant_data: Dict[str, Any], api_url: str = "http://localhost:8080/crawling/naver-places") -> bool:
    """
    레스토랑 데이터를 API로 전송합니다.
    
    Args:
        restaurant_data: API로 전송할 레스토랑 데이터
        api_url: API 엔드포인트 URL
        
    Returns:
        성공 여부 (True/False)
    """
    try:
        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
        
        response = requests.post(
            api_url,
            json=restaurant_data,
            headers=headers,
            timeout=30
        )
        
        if response.status_code == 200:
            print(f"✅ 성공: {restaurant_data['name']}")
            return True
        else:
            print(f"❌ 실패: {restaurant_data['name']} - Status: {response.status_code}")
            print(f"   응답: {response.text}")
            return False
            
    except requests.exceptions.RequestException as e:
        print(f"❌ 요청 에러: {restaurant_data['name']} - {str(e)}")
        return False
    except Exception as e:
        print(f"❌ 일반 에러: {restaurant_data['name']} - {str(e)}")
        return False

In [5]:
def process_restaurants_to_api(json_file_path: str, api_url: str = "http://localhost:8080/crawling/naver-places", delay: float = 0.5):
    """
    JSON 파일의 모든 레스토랑을 API로 전송합니다.
    
    Args:
        json_file_path: JSON 파일 경로
        api_url: API 엔드포인트 URL
        delay: 요청 간 대기 시간 (초)
    """
    print(f"📄 JSON 파일 로딩 중: {json_file_path}")
    
    # JSON 파일 로드
    restaurants = load_restaurant_data(json_file_path)
    total_restaurants = len(restaurants)
    
    print(f"📊 총 {total_restaurants}개의 레스토랑 발견")
    print(f"🎯 API 엔드포인트: {api_url}")
    print("=" * 50)
    
    success_count = 0
    failure_count = 0
    
    for i, restaurant in enumerate(restaurants, 1):
        print(f"\n[{i}/{total_restaurants}] 처리 중...")
        
        # API 형식으로 변환
        formatted_data = format_restaurant_for_api(restaurant)
        
        # 메뉴가 없는 레스토랑 스킵
        if not formatted_data['menus']:
            print(f"⚠️  스킵: {formatted_data['name']} - 메뉴 정보 없음")
            continue
        
        # API로 전송
        if send_restaurant_to_api(formatted_data, api_url):
            success_count += 1
        else:
            failure_count += 1
        
        # 요청 간 대기
        if delay > 0:
            time.sleep(delay)
    
    print("\n" + "=" * 50)
    print(f"📈 전송 완료!")
    print(f"✅ 성공: {success_count}개")
    print(f"❌ 실패: {failure_count}개")
    print(f"📊 성공률: {success_count/(success_count+failure_count)*100:.1f}%" if (success_count+failure_count) > 0 else "0%")

In [6]:
# 실행 설정
JSON_FILE_PATH = "./naver_restaurants_20250813_153837.json"  # 또는 "./naver_restaurants_20250812_012556.json"
API_URL = "http://localhost:8080/crawling/naver-places"
DELAY_BETWEEN_REQUESTS = 0.5  # 0.5초 대기

# 파일 존재 확인
if os.path.exists(JSON_FILE_PATH):
    print(f"✅ JSON 파일 확인됨: {JSON_FILE_PATH}")
else:
    print(f"❌ JSON 파일을 찾을 수 없습니다: {JSON_FILE_PATH}")
    print("현재 디렉토리의 파일들:")
    for file in os.listdir('.'):
        if file.endswith('.json'):
            print(f"  📄 {file}")

✅ JSON 파일 확인됨: ./naver_restaurants_20250813_153837.json


In [7]:
# 테스트: 첫 번째 레스토랑만 전송해보기
def test_single_restaurant(json_file_path: str):
    """
    첫 번째 레스토랑만 테스트로 전송합니다.
    """
    restaurants = load_restaurant_data(json_file_path)
    
    if not restaurants:
        print("❌ 레스토랑 데이터가 없습니다.")
        return
    
    # 메뉴가 있는 첫 번째 레스토랑 찾기
    test_restaurant = None
    for restaurant in restaurants:
        if restaurant.get('menus') and len(restaurant['menus']) > 0:
            # None 가격인 메뉴 제외하고 유효한 메뉴가 있는지 확인
            valid_menus = [m for m in restaurant['menus'] if m.get('price') is not None]
            if valid_menus:
                test_restaurant = restaurant
                break
    
    if not test_restaurant:
        print("❌ 유효한 메뉴가 있는 레스토랑을 찾을 수 없습니다.")
        return
    
    print(f"🧪 테스트 레스토랑: {test_restaurant['name']}")
    print(f"📍 주소: {test_restaurant.get('address', 'N/A')}")
    print(f"🍽️  메뉴 수: {len([m for m in test_restaurant['menus'] if m.get('price') is not None])}개")
    
    # API 형식으로 변환
    formatted_data = format_restaurant_for_api(test_restaurant)
    
    print(f"\n📤 API 요청 데이터:")
    print(json.dumps(formatted_data, ensure_ascii=False, indent=2))
    
    # API로 전송
    print(f"\n🚀 API 전송 중...")
    success = send_restaurant_to_api(formatted_data)
    
    return success

# 테스트 실행 (주석 해제하여 실행)
# test_single_restaurant(JSON_FILE_PATH)

In [8]:
# 전체 레스토랑 API 전송 실행
# ⚠️ 주의: 실행 전에 API 서버가 실행 중인지 확인하세요!

def main():
    """
    메인 실행 함수
    """
    print("🍽️  네이버 플레이스 레스토랑 → API 전송 시작")
    print("=" * 60)
    
    # 1. 단일 테스트 먼저 실행
    print("1️⃣ 단일 레스토랑 테스트 중...")
    test_success = test_single_restaurant(JSON_FILE_PATH)
    
    if not test_success:
        print("❌ 테스트 실패! API 서버 상태를 확인하세요.")
        return
    
    print("\n✅ 테스트 성공! 전체 배치 처리를 시작합니다.")
    
    # 사용자 확인
    response = input("\n계속 진행하시겠습니까? (y/N): ")
    if response.lower() != 'y':
        print("🛑 처리를 중단합니다.")
        return
    
    # 2. 전체 배치 처리
    print("\n2️⃣ 전체 레스토랑 배치 처리 중...")
    process_restaurants_to_api(
        json_file_path=JSON_FILE_PATH,
        api_url=API_URL,
        delay=DELAY_BETWEEN_REQUESTS
    )
    
    print("\n🎉 모든 처리가 완료되었습니다!")

# 실행 (주석 해제하여 실행)
# main()

In [9]:
# 🎯 사용 방법 가이드
"""
📋 실행 순서:

1. 먼저 모든 셀을 순서대로 실행하여 함수들을 로드합니다.

2. JSON 파일 경로를 확인합니다:
   - JSON_FILE_PATH 변수에서 원하는 파일을 선택
   - 두 개 파일 중 선택 가능:
     * naver_restaurants_20250812_012556.json (127개 레스토랑)
     * naver_restaurants_20250812_130001.json (52개 레스토랑)

3. API 서버가 실행 중인지 확인합니다:
   - http://localhost:8080 에서 서버 실행 필요

4. 테스트 실행:
   test_single_restaurant(JSON_FILE_PATH)

5. 전체 실행:
   main()

🔧 설정 옵션:
- DELAY_BETWEEN_REQUESTS: 요청 간 대기 시간 조정
- API_URL: API 엔드포인트 변경 가능
"""

# 추가 유틸리티: 특정 레스토랑만 전송
def send_specific_restaurants(json_file_path: str, restaurant_names: List[str]):
    """
    특정 이름의 레스토랑들만 API로 전송합니다.
    
    Args:
        json_file_path: JSON 파일 경로
        restaurant_names: 전송할 레스토랑 이름 리스트
    """
    restaurants = load_restaurant_data(json_file_path)
    
    found_restaurants = []
    for restaurant in restaurants:
        if restaurant.get('name') in restaurant_names:
            found_restaurants.append(restaurant)
    
    print(f"📊 찾은 레스토랑: {len(found_restaurants)}/{len(restaurant_names)}개")
    
    for restaurant in found_restaurants:
        formatted_data = format_restaurant_for_api(restaurant)
        if formatted_data['menus']:
            send_restaurant_to_api(formatted_data)
        else:
            print(f"⚠️  스킵: {restaurant['name']} - 메뉴 없음")

# 예시 사용법:
# send_specific_restaurants(JSON_FILE_PATH, ["자연산 해담일식 대게마을", "멕시타이거"])

In [10]:
# 테스트 실행 - 단일 레스토랑 확인
print("🧪 단일 레스토랑 테스트 실행 중...")
test_single_restaurant(JSON_FILE_PATH)

🧪 단일 레스토랑 테스트 실행 중...
🧪 테스트 레스토랑: 멕시타이거
📍 주소: 서울 강남구 테헤란로1길 42 B1층 1호
🍽️  메뉴 수: 41개

📤 API 요청 데이터:
{
  "id": "1486617466",
  "name": "멕시타이거",
  "category": "요리주점",
  "address": "서울 강남구 테헤란로1길 42 B1층 1호",
  "contact": "0507-1425-9866",
  "menus": [
    {
      "name": "지미추리 화덕치킨",
      "price": 21900
    },
    {
      "name": "하바네로 화덕치킨",
      "price": 21900
    },
    {
      "name": "타이거새우 버터몰레",
      "price": 25900
    },
    {
      "name": "버터몰레 플레이트",
      "price": 13900
    },
    {
      "name": "비리얌 라멘",
      "price": 13900
    },
    {
      "name": "치폴레 퀘사디아",
      "price": 19900
    },
    {
      "name": "망고 세비체",
      "price": 7900
    },
    {
      "name": "지미추리 치킨몰레",
      "price": 55700
    },
    {
      "name": "하바네로 치킨몰레",
      "price": 55700
    },
    {
      "name": "빠빠스",
      "price": 13900
    },
    {
      "name": "치폴레몰레 플레이트",
      "price": 13900
    },
    {
      "name": "피쉬 타코",
      "price": 18900
    },
    {
      "name": "호세 데낄라 하이볼",
  

True

In [11]:
# 🚀 전체 레스토랑 API 전송 실행
# ⚠️ 주의: 아래 주석을 해제하고 실행하면 모든 레스토랑이 API로 전송됩니다!

main()

print("📋 전체 실행을 원하시면 위의 main() 주석을 해제하고 실행하세요.")
print(f"📊 현재 JSON 파일: {JSON_FILE_PATH}")
print(f"🎯 API 엔드포인트: {API_URL}")

# 또는 특정 레스토랑만 전송하고 싶다면:
# send_specific_restaurants(JSON_FILE_PATH, ["자연산 해담일식 대게마을", "강삼가든"])

🍽️  네이버 플레이스 레스토랑 → API 전송 시작
1️⃣ 단일 레스토랑 테스트 중...
🧪 테스트 레스토랑: 멕시타이거
📍 주소: 서울 강남구 테헤란로1길 42 B1층 1호
🍽️  메뉴 수: 41개

📤 API 요청 데이터:
{
  "id": "1486617466",
  "name": "멕시타이거",
  "category": "요리주점",
  "address": "서울 강남구 테헤란로1길 42 B1층 1호",
  "contact": "0507-1425-9866",
  "menus": [
    {
      "name": "지미추리 화덕치킨",
      "price": 21900
    },
    {
      "name": "하바네로 화덕치킨",
      "price": 21900
    },
    {
      "name": "타이거새우 버터몰레",
      "price": 25900
    },
    {
      "name": "버터몰레 플레이트",
      "price": 13900
    },
    {
      "name": "비리얌 라멘",
      "price": 13900
    },
    {
      "name": "치폴레 퀘사디아",
      "price": 19900
    },
    {
      "name": "망고 세비체",
      "price": 7900
    },
    {
      "name": "지미추리 치킨몰레",
      "price": 55700
    },
    {
      "name": "하바네로 치킨몰레",
      "price": 55700
    },
    {
      "name": "빠빠스",
      "price": 13900
    },
    {
      "name": "치폴레몰레 플레이트",
      "price": 13900
    },
    {
      "name": "피쉬 타코",
      "price": 18900
    },
    {
 